In [105]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import ipywidgets as widgets
from IPython.display import display

In [99]:
from IPython.display import display, HTML

display(HTML("""
<style>
    .widget-label { 
        font-size: 24px !important;
    }
    .slider-value { 
        font-size: 24px !important;
    }
    .widget-hslider .widget-readout {
        font-size: 22px !important;
    }
</style>
"""))

In [186]:
grid_size = 50000
circle_radius = 10000

def update_circle(angle_deg):
    angle_rad = np.radians(angle_deg)
    
    circle_x = circle_radius * np.cos(angle_rad)
    circle_y = circle_radius * np.sin(angle_rad)
    
    fig = plt.figure(figsize=(12, 5))
    gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], wspace=0.3) 

    cur_ax = plt.subplot(gs[0, 0])
    cur_ax.set_xlim(-grid_size / 2, grid_size / 2)
    cur_ax.set_ylim(-grid_size / 2, grid_size / 2)
    cur_ax.scatter(circle_x, circle_y, color='red', s=10, label=f"Source: ({circle_x:.2f}, {circle_y:.2f})")

    r1x = 0
    r1y = -1000
    dist_from_source_to_r1 = max(1, np.sqrt(((circle_x-r1x)**2) + ((circle_y-r1y)**2)))
    transmission_loss_to_r1 = 20*np.log10(dist_from_source_to_r1)
    cur_ax.scatter(r1x, r1y, color='green', s=10, label=f"Receiver #1 {dist_from_source_to_r1:.2f}m from source")

    r1x = 0
    r1y = 1000
    dist_from_source_to_r2 = max(1, np.sqrt(((circle_x-r1x)**2) + ((circle_y-r1y)**2)))
    transmission_loss_to_r2 = 20*np.log10(dist_from_source_to_r2)
    cur_ax.scatter(r1x, r1y, color='blue', s=10, label=f"Receiver #2 {dist_from_source_to_r2:.2f}m from source")
    cur_ax.set_xlabel("X (meters)")
    cur_ax.set_ylabel("Y (meters)")
    cur_ax.set_title(f"Circle Position (Angle = {angle_deg}°)")
    cur_ax.legend()
    
    sampling_rate = 2000
    t = np.linspace(0, 120, sampling_rate*120)
    signal = np.sin(t*np.pi)

    start = 10
    duration = 5
    end = start+duration

    speed_of_sound_in_water = 1500
    time_to_r1 = dist_from_source_to_r1 / speed_of_sound_in_water
    time_to_r2 = dist_from_source_to_r2 / speed_of_sound_in_water

    gs_right = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec=gs[0, 1], hspace=0.8)

    ax_r1 = plt.subplot(gs_right[0])
    receive_signal1 = signal/(dist_from_source_to_r1**2)
    receive_signal2 = signal/(dist_from_source_to_r2**2)
    ax_r1.plot(t+time_to_r1, receive_signal1, 'green')
    ax_r1.plot(t+time_to_r2, receive_signal2, 'blue')

    chunk_of_received_signal1_in_window = receive_signal1[int((start-time_to_r1)*sampling_rate):int((end-time_to_r1)*sampling_rate)]
    chunk_of_received_signal2_in_window = receive_signal2[int((start-time_to_r2)*sampling_rate):int((end-time_to_r2)*sampling_rate)]


    ax_r1.plot(t[int((start)*sampling_rate):int((end)*sampling_rate)], chunk_of_received_signal1_in_window+chunk_of_received_signal2_in_window, 'k')
    ax_r1.set_xlabel("Time (s)")
    ax_r1.set_ylabel("Amplitude")
    ax_r1.set_title("Received Signal #1")
    ax_r1.grid(which='both')
    ax_r1.set_xlim(start, end)
    ax_r1.set_ylim(-1e-7, 1e-7)

    # ax_r2 = plt.subplot(gs_right[1])
    # receive_signal2 = signal/(dist_from_source_to_r2**2)
    # ax_r2.plot(t+time_to_r2, receive_signal2, 'blue')
    # ax_r2.set_xlabel("Time (s)")
    # ax_r2.set_ylabel("Amplitude")
    # ax_r2.set_title("Received Signal #2")
    # ax_r2.grid(which='both')
    # ax_r2.set_xlim(start_time_of_transmission, start_time_of_transmission+duration_of_transmission)

    # ax_r1 = plt.subplot(gs_right[2])
    # ax_r1.plot(t, receive_signal2+receive_signal1, 'green')
    # ax_r1.set_xlabel("Time (s)")
    # ax_r1.set_ylabel("Amplitude")
    # ax_r1.set_title("Received Signal #1")
    # ax_r1.grid(which='both')
    # ax_r1.set_xlim(start_time_of_transmission, start_time_of_transmission+duration_of_transmission)

    plt.show()


In [187]:
angle_slider = widgets.IntSlider(
    value=0, min=0, max=360, step=1, description="Angle (°)", continuous_update=True, style={'description_width': 'initial'}, layout=widgets.Layout(width="600px")
)

interactive_plot = widgets.interactive(update_circle, angle_deg=angle_slider)
display(interactive_plot)


interactive(children=(IntSlider(value=0, description='Angle (°)', layout=Layout(width='600px'), max=360, style…